In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")
from tqdm.notebook import tqdm
import time
from IPython.display import clear_output

import ee, geemap

In [2]:
%load_ext autoreload
%autoreload 2
import sys  
sys.path.insert(0, '../lib/')

In [3]:
%reload_ext autoreload
import sand_classification as sc
import quality_mosaic as qm

# Part I: Get the locations from Google Drive

In [4]:
sheet_id = '13nF_pJ02Bd70cDJamuKbvZIkIdJ-kOI4O3Cx9K7Wzos'
# sheet_id = '1XqOtPkiE_Q0dfGSoyxrH730RkwrTczcRbDeJJpqRByQ'
sheet_name = 'nonsand'
drive_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
drive_url

'https://docs.google.com/spreadsheets/d/13nF_pJ02Bd70cDJamuKbvZIkIdJ-kOI4O3Cx9K7Wzos/gviz/tq?tqx=out:csv&sheet=nonsand'

In [5]:
output_file_path = '../data/labels/gt-bands.xlsx'

In [6]:
#Read in a google sheets with the ground truth data from string drive_loc
df = pd.read_csv(drive_url)
df

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes,Unnamed: 9,...,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27
0,ns-0,25.449886,82.858051,bare,2022-10-03,Airport,Author,India,Varanasi Airport,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ns-1,25.230858,83.021355,greenveg,2022-02-19,Vegetated Fields,Author,India,Off Ganga,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ns-2,25.246509,83.025825,bare,2022-02-19,Bare fields,Author,India,Off Ganga,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ns-3,25.208222,82.980644,water,2022-02-19,River water,Author,India,Ganga,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ns-4,22.573816,88.349404,bare,2022-01-02,"Urban scene, kolkata",Author,India,Off Hoogly,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,ns-5,22.642989,88.439406,bare,2020-05-11,Airport building,Author,India,Kolkata airport,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ns-6,19.752572,85.702946,whitewater,2020-09-08,Off Puri,Author,India,Puri beach,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,ns-7,19.744671,85.694326,water,2020-09-08,Off Puri,Author,India,Puri beach,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,ns-8,19.762033,85.703707,greenveg,2020-09-08,Off Puri,Author,India,Puri beach,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,ns-9,-23.855739,35.552899,whitewater,2020-01-02,Off Tofo Beach,Author,Mozambique,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#look for any NaN values in the column 'date' or 'Longitude' or 'Latitude' in df_merged and drop them
df = df.dropna(subset=['Date', 'Longitude', 'Latitude']).reset_index(drop=True)
#drop any columns that have 'Unnamed:' in the column name
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes
0,ns-0,25.449886,82.858051,bare,2022-10-03,Airport,Author,India,Varanasi Airport
1,ns-1,25.230858,83.021355,greenveg,2022-02-19,Vegetated Fields,Author,India,Off Ganga
2,ns-2,25.246509,83.025825,bare,2022-02-19,Bare fields,Author,India,Off Ganga
3,ns-3,25.208222,82.980644,water,2022-02-19,River water,Author,India,Ganga
4,ns-4,22.573816,88.349404,bare,2022-01-02,"Urban scene, kolkata",Author,India,Off Hoogly
5,ns-5,22.642989,88.439406,bare,2020-05-11,Airport building,Author,India,Kolkata airport
6,ns-6,19.752572,85.702946,whitewater,2020-09-08,Off Puri,Author,India,Puri beach
7,ns-7,19.744671,85.694326,water,2020-09-08,Off Puri,Author,India,Puri beach
8,ns-8,19.762033,85.703707,greenveg,2020-09-08,Off Puri,Author,India,Puri beach
9,ns-9,-23.855739,35.552899,whitewater,2020-01-02,Off Tofo Beach,Author,Mozambique,NaN


In [8]:
assert(len(df[df['Date'].isna() | df['Longitude'].isna() | df['Latitude'].isna()]) == 0)

# Part II: Select subset for training

Here we iterate over the df and iteratively pick and adjust the locations we want to keep

In [9]:
df.columns

Index(['ID', 'Latitude', 'Longitude', 'Class', 'Date', 'Site', 'Source',
       'Country', 'Notes'],
      dtype='object')

In [12]:
# ee.Authenticate()
ee.Initialize()



Successfully saved authorization token.


In [13]:
output = df.copy()
output

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes
0,ns-0,25.449886,82.858051,bare,2022-10-03,Airport,Author,India,Varanasi Airport
1,ns-1,25.230858,83.021355,greenveg,2022-02-19,Vegetated Fields,Author,India,Off Ganga
2,ns-2,25.246509,83.025825,bare,2022-02-19,Bare fields,Author,India,Off Ganga
3,ns-3,25.208222,82.980644,water,2022-02-19,River water,Author,India,Ganga
4,ns-4,22.573816,88.349404,bare,2022-01-02,"Urban scene, kolkata",Author,India,Off Hoogly
5,ns-5,22.642989,88.439406,bare,2020-05-11,Airport building,Author,India,Kolkata airport
6,ns-6,19.752572,85.702946,whitewater,2020-09-08,Off Puri,Author,India,Puri beach
7,ns-7,19.744671,85.694326,water,2020-09-08,Off Puri,Author,India,Puri beach
8,ns-8,19.762033,85.703707,greenveg,2020-09-08,Off Puri,Author,India,Puri beach
9,ns-9,-23.855739,35.552899,whitewater,2020-01-02,Off Tofo Beach,Author,Mozambique,NaN


In [14]:
%reload_ext autoreload
import sand_classification as sc
import quality_mosaic as qm
import file_management as fm

In [81]:
INDEX = 6

In [70]:
sampling_buffer_m = 5
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Manual Loop

Keep running the following 2 cells until you're done with the manual sampling.


In [85]:
#Uncomment to change the date of the image
output['Date'].iloc[INDEX] = '2021-07-11'

In [92]:
s1_s2, sample = qm.get_s1s2_data(output, Map, INDEX, display_smap=False, mosaic_method='qm', sampling_buffer_m=sampling_buffer_m, 
                                 max_search_window_months=3, median_samples=5,roi_buffer_m=5000, obia=True)

Index:  8  ID:  ns-8 Class:  greenveg  Site:  Off Puri
Search window from 2020-06-08 to 2020-12-08


## Use this immediately after the map is refreshed to either select or discard the sample

INDEX will auto update on calling this function.
It will also overwrite just the sheet if the same exists, but will preserve the other sheets in the book

In [ ]:
output, INDEX = qm.get_training_sample(output, s1_s2, sample, Map, INDEX, display_clusters=False, obia=True)
# fm.write_to_excel(output, output_file_path, sheet_name)
with pd.ExcelWriter(output_file_path,engine="openpyxl",mode="a",if_sheet_exists="replace") as writer:
    output.to_excel(writer,sheet_name=sheet_name,index=False)
output.iloc[INDEX-1]

## Final Output

In [89]:
output

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes,B2_mean,...,B11_mean,B12_mean,VV_mean,VH_mean,mTGSI_mean,BSI_mean,NDWI_mean,keep,location_tweaked,class_code
0,ns-0,25.449886,82.858051,bare,2022-10-03,Airport,Author,India,Varanasi Airport,2079.636475,...,3911.121094,3898.121094,-16.306322,-23.792824,0.093876,0.114805,-0.127118,True,False,5
1,ns-1,25.230677,83.021533,greenveg,2020-12-11,Vegetated Fields,Author,India,Off Ganga,438.198212,...,2346.062988,1156.909912,-9.820938,-15.811003,-0.413501,-0.234269,-0.656859,True,True,6
2,ns-2,25.246734,83.025929,bare,2022-02-19,Bare fields,Author,India,Off Ganga,1274.894165,...,2501.282471,2008.023560,-13.695921,-20.938858,-0.035784,0.027038,-0.261934,True,True,5
3,ns-3,25.208222,82.980644,water,2022-02-19,River water,Author,India,Ganga,296.329620,...,125.648048,122.273743,-25.812956,-27.554104,-0.125209,-0.175154,0.247106,True,False,4
4,ns-4,22.573932,88.349081,bare,2022-01-02,"Urban scene, kolkata",Author,India,Off Hoogly,655.157043,...,1108.735596,895.066101,-6.657454,-13.574802,-0.001648,0.059572,-0.142189,True,True,5
5,ns-5,22.642989,88.439406,bare,2020-05-11,Airport building,Author,India,Kolkata airport,4966.920898,...,7159.294922,7827.028809,7.744934,-1.827149,0.107236,0.109206,0.001565,True,False,5
6,ns-6,19.751838,85.702412,whitewater,2021-07-11,Off Puri,Author,India,Puri beach,1990.053101,...,1342.601807,1110.654907,-14.995195,-25.048250,-0.046942,-0.028729,0.082091,True,True,3
7,ns-7,19.744671,85.694326,water,2020-09-08,Off Puri,Author,India,Puri beach,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
8,ns-8,19.762033,85.703707,greenveg,2020-09-08,Off Puri,Author,India,Puri beach,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
9,ns-9,-23.855629,35.553179,whitewater,2020-01-02,Off Tofo Beach,Author,Mozambique,NaN,3402.970215,...,814.380615,638.238831,-17.386923,-27.102491,-0.129007,-0.159450,0.296712,True,True,3
